In [1]:
import z3

In [82]:
window_size = 2
nb_res = 1
num_burst_buffers = 1

# Number of requested compute nodes
# N = [16, 1, 2, 3]
N = [1, 1]
# Size of requested burst buffer per compute node rounded up to kB
# B = [20000000, 884041, 2446271, 609585]
B = [1, 1]

# i-th job in the window is selected to be scheduled
a = [z3.Bool('a_{}'.format(i)) for i in range(window_size)]
# j-th compute node is assigned to i-th job
c = [[z3.Bool('c_{}_{}'.format(i, j)) for j in range(nb_res)]
     for i in range(window_size)]
# k-th storage node is assigned to j-th compute node
b = [[z3.Bool('b_{}_{}'.format(j, k)) for k in range(num_burst_buffers)]
     for j in range(nb_res)]

# z3.set_param('parallel.enable', True)
o = z3.Optimize()
o.set(priority='lex')

# First job from the queue must always be selected
o.add(a[0] == True)
# Each application must receive requested number of nodes if selected
for i in range(window_size):
    proc_count = [(c[i][j], 1) for j in range(nb_res)]
    all_proc_not = [z3.Not(c[i][j]) for j in range(nb_res)]
    o.add(z3.If(a[i], z3.PbEq(proc_count, N[i]), z3.And(all_proc_not)))
# Each compute node can be assigned to only one application
for j in range(nb_res):
    o.add(z3.PbLe([(c[i][j], 1) for i in range(window_size)], 1))
# Each selected compute node is assigned with exactly one storage node
for j in range(nb_res):
    any_proc = [c[i][j] for i in range(window_size)]
    bb_count = [(b[j][k], 1) for k in range(num_burst_buffers)]
    all_bb_not = [z3.Not(b[j][k]) for k in range(num_burst_buffers)]
    o.add(z3.If(z3.Or(any_proc), z3.PbEq(storage_count, 1), z3.And(all_bb_not)))
# Limit burst buffer assignments by the available space
for k in range(num_burst_buffers):
    storage_sum = [(z3.And(c[i][j], b[j][k]), B[i])
                   for i in range(window_size) for j in range(nb_res)]
    # Available burst buffer space rounded down to kB
    BBAvail = 1
    o.add(z3.PbLe(storage_sum, BBAvail))

# Maximise compute utilisation
compute = o.maximize(z3.Sum([z3.If(a[i], N[i], 0) for i in range(window_size)]))
# Maximise burst buffer utilisation
storage = o.maximize(z3.Sum([z3.If(a[i], N[i] * B[i], 0) for i in range(window_size)]))

In [83]:
o

(set-option :opt.priority lex)
(declare-fun a_0 () Bool)
(declare-fun c_0_0 () Bool)
(declare-fun c_1_0 () Bool)
(declare-fun a_1 () Bool)
(declare-fun b_0_0 () Bool)
(assert (= a_0 true))
(assert (ite a_0 ((_ pbeq 1 1) c_0_0) (and (not c_0_0))))
(assert (ite a_1 ((_ pbeq 1 1) c_1_0) (and (not c_1_0))))
(assert ((_ at-most 1) c_0_0 c_1_0))
(assert (ite (or c_0_0 c_1_0) ((_ pbeq 1 1) b_0_0) (and (not b_0_0))))
(assert ((_ at-most 1) (and c_0_0 b_0_0) (and c_1_0 b_0_0)))
(maximize (+ (ite a_0 1 0) (ite a_1 1 0)))
(maximize (+ (ite a_0 1 0) (ite a_1 1 0)))
(check-sat)

In [84]:
o.check()

sat

In [85]:
o.unsat_core()

[]

In [86]:
o.model()

[b_0_0 = True,
 c_1_0 = False,
 a_1 = False,
 c_0_0 = True,
 a_0 = True]

In [87]:
compute.value()

1

In [88]:
storage.value()

1

In [89]:
o.statistics()

(:eliminated-vars 5
 :max-memory      10.01
 :memory          10.00
 :num-allocs      1996318
 :rlimit-count    2726311
 :sat-decisions   1
 :sat-mk-var      1)

In [90]:
o.help()

ctrl_c (bool) enable interrupts from ctrl-c (default: true)
dump_benchmarks (bool) dump benchmarks for profiling (default: false)
dump_models (bool) display intermediary models to stdout (default: false)
elim_01 (bool) eliminate 01 variables (default: true)
enable_sat (bool) enable the new SAT core for propositional constraints (default: true)
enable_sls (bool) enable SLS tuning during weighted maxsast (default: false)
maxlex.enable (bool) enable maxlex heuristic for lexicographic MaxSAT problems (default: true)
maxres.add_upper_bound_block (bool) restict upper bound with constraint (default: false)
maxres.hill_climb (bool) give preference for large weight cores (default: true)
maxres.max_core_size (unsigned int) break batch of generated cores if size reaches this number (default: 3)
maxres.max_correction_set_size (unsigned int) allow generating correction set constraints up to maximal size (default: 3)
maxres.max_num_cores (unsigned int) maximal number of cores per round (default: 429